In [10]:
### Import Statements ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.linear_model import LogisticRegressionCV
from scipy.sparse import coo_matrix, hstack
import scipy

In [27]:
df = pd.read_csv("train.csv")
df2 = pd.read_csv("test.csv")

In [50]:
df

,id,text,MWS,HPL,EAP
0,id26305,"This process, however, afforded me no means of...",0,0,1
1,id17569,It never once occurred to me that the fumbling...,0,1,0
2,id11008,"In his left hand was a gold snuff box, from wh...",0,0,1
3,id27763,How lovely is spring As we looked from Windsor...,1,0,0
4,id12958,"Finding nothing else, not even gold, the Super...",0,1,0
5,id22965,"A youth passed in solitude, my best years spen...",1,0,0
6,id09674,"The astronomer, perhaps, at this point, took r...",0,0,1
7,id13515,The surcingle hung in ribands from my body.,0,0,1
8,id19322,I knew that you could not say to yourself 'ste...,0,0,1
9,id00912,I confess that neither the structure of langua...,1,0,0


In [31]:
def convert_(feature):
    if feature[0] == 'EAP':
        return 1
    else:
        return 0
df['EAP'] = df[['author']].apply(convert_,axis=1)

In [32]:
df.drop('author',axis=1,inplace=True)

In [33]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['text'])

In [34]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [44]:
X_new_counts = count_vect.transform(df2['text'])
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
X_new_tfidf.shape

(8392, 25068)

In [46]:
for i in ['EAP','MWS','HPL']:
    clf = LogisticRegression().fit(X_train_tfidf, df[i])
    exec("pred_%s = pd.Series(clf.predict_proba(X_new_tfidf).flatten()[1::2])" % i)
    print(i)

EAP
MWS
HPL


In [47]:
pred_EA

(8392,)

In [48]:
submission = pd.DataFrame({
             'EAP': pred_EAP,
             'HPL': pred_HPL,
             'MWS': pred_MWS,
             'id': df2['id'],
             })

In [49]:
submission.to_csv('submission.csv',index=False)